In [1]:
#uruchomienie sesji SPARKA
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1674289536756_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK BADGES.XML__

In [2]:
#ustawienie odpowiednich typów danych dla pliku Badges.xml
from pyspark.sql.types import *

schemaBadges = StructType([
    StructField("_Class", IntegerType(), True),
    StructField("_Date", TimestampType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_Name", StringType(), True),
    StructField("_TagBased", BooleanType(), True),
    StructField("_UserId", IntegerType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#wczytanie pliku Badges.xml z pomocą modułu spark-xml do DataFrame'u
dfBadges = spark.read.format('xml').option('rootTag', 'badges').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Badges.xml',schema=schemaBadges)
dfBadges.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _Class: integer (nullable = true)
 |-- _Date: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _Name: string (nullable = true)
 |-- _TagBased: boolean (nullable = true)
 |-- _UserId: integer (nullable = true)

In [4]:
dfBadges.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+---+--------------+---------+-------+
|_Class|               _Date|_Id|         _Name|_TagBased|_UserId|
+------+--------------------+---+--------------+---------+-------+
|     3|2014-01-21 20:52:...|  1|Autobiographer|    false|      1|
|     3|2014-01-21 20:52:...|  2|Autobiographer|    false|      2|
|     3|2014-01-21 20:52:...|  3|Autobiographer|    false|      6|
|     3|2014-01-21 20:52:...|  4|Autobiographer|    false|      7|
|     3|2014-01-21 20:52:...|  5|Autobiographer|    false|      9|
+------+--------------------+---+--------------+---------+-------+
only showing top 5 rows

In [5]:
#zmiana nazwy kolumn
dfBadges1 = dfBadges.withColumnRenamed("_Class","Class")\
                    .withColumnRenamed("_Date","Date")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_Name","Name")\
                    .withColumnRenamed("_TagBased","TagBased")\
                    .withColumnRenamed("_UserId","UserId")
dfBadges1.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+---+--------------+--------+------+
|Class|                Date| Id|          Name|TagBased|UserId|
+-----+--------------------+---+--------------+--------+------+
|    3|2014-01-21 20:52:...|  1|Autobiographer|   false|     1|
|    3|2014-01-21 20:52:...|  2|Autobiographer|   false|     2|
|    3|2014-01-21 20:52:...|  3|Autobiographer|   false|     6|
|    3|2014-01-21 20:52:...|  4|Autobiographer|   false|     7|
|    3|2014-01-21 20:52:...|  5|Autobiographer|   false|     9|
+-----+--------------------+---+--------------+--------+------+
only showing top 5 rows

In [7]:
#rozbicie pierwotnej kolumny Date na datę oraz czas
from pyspark.sql.functions import split

dfBadges2 = dfBadges1.select("Class", "Date", "Id", "Name", "TagBased", "UserId").withColumn('Date1',split(dfBadges1.Date, ' ')[0])\
                                                                                 .withColumn('Time',split(dfBadges1.Date,' ')[1])\
                                                                                 .drop(dfBadges1.Date)
dfBadges2.show(5)
dfBadges2.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+--------------+--------+------+----------+-----------+
|Class| Id|          Name|TagBased|UserId|     Date1|       Time|
+-----+---+--------------+--------+------+----------+-----------+
|    3|  1|Autobiographer|   false|     1|2014-01-21|20:52:16.97|
|    3|  2|Autobiographer|   false|     2|2014-01-21|20:52:16.97|
|    3|  3|Autobiographer|   false|     6|2014-01-21|20:52:16.97|
|    3|  4|Autobiographer|   false|     7|2014-01-21|20:52:16.97|
|    3|  5|Autobiographer|   false|     9|2014-01-21|20:52:16.97|
+-----+---+--------------+--------+------+----------+-----------+
only showing top 5 rows

<bound method DataFrame.printSchema of DataFrame[Class: int, Id: int, Name: string, TagBased: boolean, UserId: int, Date1: string, Time: string]>

In [8]:
#ustawienie odpowiedniego Data Type dla nowej kolumny Date, ze względu na brak typu: time w Sparku, dane w kolumnie Time pozostały jako string
dfBadges3 = dfBadges2.withColumn("Date1", dfBadges2.Date1.cast(DateType()))
dfBadges3.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+--------------+--------+------+----------+-----------+
|Class| Id|          Name|TagBased|UserId|     Date1|       Time|
+-----+---+--------------+--------+------+----------+-----------+
|    3|  1|Autobiographer|   false|     1|2014-01-21|20:52:16.97|
|    3|  2|Autobiographer|   false|     2|2014-01-21|20:52:16.97|
|    3|  3|Autobiographer|   false|     6|2014-01-21|20:52:16.97|
|    3|  4|Autobiographer|   false|     7|2014-01-21|20:52:16.97|
+-----+---+--------------+--------+------+----------+-----------+
only showing top 4 rows

In [9]:
dfBadges4 = dfBadges3.withColumnRenamed("Date1", "Date")
dfBadges4.show(5)
dfBadges4.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+--------------+--------+------+----------+-----------+
|Class| Id|          Name|TagBased|UserId|      Date|       Time|
+-----+---+--------------+--------+------+----------+-----------+
|    3|  1|Autobiographer|   false|     1|2014-01-21|20:52:16.97|
|    3|  2|Autobiographer|   false|     2|2014-01-21|20:52:16.97|
|    3|  3|Autobiographer|   false|     6|2014-01-21|20:52:16.97|
|    3|  4|Autobiographer|   false|     7|2014-01-21|20:52:16.97|
|    3|  5|Autobiographer|   false|     9|2014-01-21|20:52:16.97|
+-----+---+--------------+--------+------+----------+-----------+
only showing top 5 rows

root
 |-- Class: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- TagBased: boolean (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)

In [10]:
#zapisanie DataFrame'u w formacie parquet w S3
dfBadges4.write.parquet('s3://stackexchangebeer1/parquet/badges.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK VOTES.XML__

In [11]:
#ustawienie odpowiednich typów danych dla pliku Votes.xml

schemaVotes = StructType([
    StructField("_BountyAmount", IntegerType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_PostId", StringType(), True),
    StructField("_UserId", BooleanType(), True),
    StructField("_VoteTypeId", IntegerType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#wczytanie pliku Badges.xml z pomocą modułu spark-xml do DataFrame'u
dfVotes = spark.read.format('xml').option('rootTag', 'votes').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Votes.xml',schema=schemaVotes)
dfVotes.printSchema()
dfVotes.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _BountyAmount: integer (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _PostId: string (nullable = true)
 |-- _UserId: boolean (nullable = true)
 |-- _VoteTypeId: integer (nullable = true)

+-------------+-------------------+---+-------+-------+-----------+
|_BountyAmount|      _CreationDate|_Id|_PostId|_UserId|_VoteTypeId|
+-------------+-------------------+---+-------+-------+-----------+
|         null|2014-01-21 00:00:00|  1|      1|   null|          2|
|         null|2014-01-21 00:00:00|  2|      1|   null|          2|
|         null|2014-01-21 00:00:00|  3|      4|   null|          2|
|         null|2014-01-21 00:00:00|  4|      1|   null|          2|
|         null|2014-01-21 00:00:00|  5|      4|   null|          2|
|         null|2014-01-21 00:00:00|  6|      3|   null|          2|
|         null|2014-01-21 00:00:00|  7|      4|   null|          2|
|         null|2014-01-21 00:00:00|  8|      1|   null|      

In [17]:
#nadanie odpowiednich nazw kolumnom
dfVotes1 = dfVotes.withColumnRenamed("_BountyAmount","BountyAmount")\
                    .withColumnRenamed("_CreationDate","Date")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_PostId","PostId")\
                    .withColumnRenamed("_UserId","UserId")\
                    .withColumnRenamed("_VoteTypeId","VoteTypeId")
dfVotes1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BountyAmount: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: string (nullable = true)
 |-- UserId: boolean (nullable = true)
 |-- VoteTypeId: integer (nullable = true)

In [33]:
#rozbicie kolumny Date na 2 kolumny z datą oraz czasem utworzenia
dfVotes2 = dfVotes1.select("BountyAmount", "Date", "Id", "PostId", "UserId", "VoteTypeId").withColumn('CreateDate',split(dfVotes1.Date, ' ')[0])\
                                                                                                  .withColumn('Time',split(dfVotes1.Date, ' ')[1])\
                                                                                                  .drop(dfVotes1.Date)
dfVotes2.show(5)
dfVotes2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---+------+------+----------+----------+--------+
|BountyAmount| Id|PostId|UserId|VoteTypeId|CreateDate|    Time|
+------------+---+------+------+----------+----------+--------+
|        null|  1|     1|  null|         2|2014-01-21|00:00:00|
|        null|  2|     1|  null|         2|2014-01-21|00:00:00|
|        null|  3|     4|  null|         2|2014-01-21|00:00:00|
|        null|  4|     1|  null|         2|2014-01-21|00:00:00|
|        null|  5|     4|  null|         2|2014-01-21|00:00:00|
+------------+---+------+------+----------+----------+--------+
only showing top 5 rows

root
 |-- BountyAmount: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: string (nullable = true)
 |-- UserId: boolean (nullable = true)
 |-- VoteTypeId: integer (nullable = true)
 |-- CreateDate: string (nullable = true)
 |-- Time: string (nullable = true)

In [49]:
#sprawdzenie ilości wierszy występujących w tabeli
rowscount = dfVotes2.count()
print(f'Votes DF rows count {rowscount}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Votes DF rows count 20453

In [46]:
#sprawdzenie kolumny BountyAmount
dfVotes2.groupBy("BountyAmount").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|BountyAmount|count|
+------------+-----+
|         100|   11|
|         200|    1|
|          50|   16|
|          25|    1|
|        null|20424|
+------------+-----+

In [47]:
#sprawdzenie kolumny UserId / w kolumnie wystepują tylko wartości null (kolumnę można usunąć)
dfVotes2.groupBy("UserId").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+
|UserId|count|
+------+-----+
|  null|20453|
+------+-----+

In [48]:
#sprawdzenie kolumny Time / w kolumnie wystepują tylko 00:00:00 (kolumnę można usunąć)
dfVotes2.groupBy("Time").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+
|    Time|count|
+--------+-----+
|00:00:00|20453|
+--------+-----+

In [ ]:
#usunięcie z tabeli dwóch kolumn Time oraz UserId 

In [59]:
dfVotes3 = dfVotes2.drop(dfVotes2["Time"]).drop(dfVotes2["UserId"])
dfVotes3.show()
dfVotes3.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---+------+----------+----------+
|BountyAmount| Id|PostId|VoteTypeId|CreateDate|
+------------+---+------+----------+----------+
|        null|  1|     1|         2|2014-01-21|
|        null|  2|     1|         2|2014-01-21|
|        null|  3|     4|         2|2014-01-21|
|        null|  4|     1|         2|2014-01-21|
|        null|  5|     4|         2|2014-01-21|
|        null|  6|     3|         2|2014-01-21|
|        null|  7|     4|         2|2014-01-21|
|        null|  8|     1|         2|2014-01-21|
|        null|  9|     8|         2|2014-01-21|
|        null| 10|     6|         2|2014-01-21|
|        null| 11|     3|         2|2014-01-21|
|        null| 12|    15|         2|2014-01-21|
|        null| 13|    14|         2|2014-01-21|
|        null| 14|     8|         2|2014-01-21|
|        null| 15|     8|         2|2014-01-21|
|        null| 16|     8|         2|2014-01-21|
|        null| 17|    18|         2|2014-01-21|
|        null| 18|    22|         2|2014

In [65]:
#zmiana typu danych kolumny CreateDate na DateType
dfVotes4 = dfVotes3.withColumn("CreateDate", dfVotes2["CreateDate"].cast(DateType()))
dfVotes4.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BountyAmount: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: string (nullable = true)
 |-- VoteTypeId: integer (nullable = true)
 |-- CreateDate: date (nullable = true)

In [66]:
#zapisanie tabeli dfVotes3 w formacie parquet
dfVotes4.write.parquet('s3://stackexchangebeer1/parquet/votes.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK POSTLINKS.XML__

In [56]:
#ustawienie odpowiednich typów danych dla pliku PostLinks.xml

schemaPostLinks = StructType([
    StructField("_CreationDate", TimestampType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_LinkTypeId", IntegerType(), True),
    StructField("_PostId", IntegerType(), True),
    StructField("_RelatedPostId", IntegerType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#wczytanie pliku PostLinks.xml z pomocą modułu spark-xml do DataFrame'u
dfPostLinks = spark.read.format('xml').option('rootTag', 'postlinks').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/PostLinks.xml',schema=schemaPostLinks)
dfPostLinks.printSchema()
dfPostLinks.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _LinkTypeId: integer (nullable = true)
 |-- _PostId: integer (nullable = true)
 |-- _RelatedPostId: integer (nullable = true)

+--------------------+---+-----------+-------+--------------+
|       _CreationDate|_Id|_LinkTypeId|_PostId|_RelatedPostId|
+--------------------+---+-----------+-------+--------------+
|2014-01-21 21:04:...| 25|          3|     29|            25|
|2014-01-21 21:42:...| 89|          1|     83|            50|
|2014-01-21 21:50:...| 95|          1|     86|             2|
|2014-01-21 22:07:...|101|          3|     47|            99|
|2014-01-21 22:13:...|102|          1|     74|             3|
+--------------------+---+-----------+-------+--------------+
only showing top 5 rows

In [58]:
#nadanie odpowiednich nazw kolumnom
dfPostLinks1 = dfPostLinks.withColumnRenamed("_CreationDate","Date")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_PostId","PostId")\
                    .withColumnRenamed("_LinkTypeId","LinkTypeId")\
                    .withColumnRenamed("_RelatedPostId","RelatedPostId")
dfPostLinks1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: timestamp (nullable = true)
 |-- Id: integer (nullable = true)
 |-- LinkTypeId: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- RelatedPostId: integer (nullable = true)

In [64]:
#rozbicie kolumny CreationDate na 2 kolumny z datą oraz czasem utworzenia
dfPostLinks2 = dfPostLinks1.select("Date", "Id", "LinkTypeId", "PostId", "RelatedPostId").withColumn('CreationDate',split(dfPostLinks1["Date"], ' ')[0])\
                                                                                                  .withColumn('Time',split(dfPostLinks1["Date"], ' ')[1])\
                                                                                                  .drop(dfPostLinks1["Date"])
dfPostLinks2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: integer (nullable = true)
 |-- LinkTypeId: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- RelatedPostId: integer (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Time: string (nullable = true)

In [67]:
#zmiana typu danych kolumny CreationDate na DateType
dfPostLinks3= dfPostLinks2.withColumn("CreationDate", dfPostLinks2["CreationDate"].cast(DateType()))
dfPostLinks3.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: integer (nullable = true)
 |-- LinkTypeId: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- RelatedPostId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Time: string (nullable = true)

In [68]:
#zapisanie tabeli dfPostLinks3 w formacie parquet
dfPostLinks3.write.parquet('s3://stackexchangebeer1/parquet/postlinks.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK TAGS.XML__

In [70]:
#ustawienie odpowiednich typów danych dla pliku Tags.xml
from pyspark.sql.types import *

schemaTags = StructType([
    StructField("_Count", IntegerType(), True),
    StructField("_ExcerptPostId", IntegerType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_TagName", StringType(), True),
    StructField("_WikiPostId", IntegerType(), True),])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
#wczytanie pliku Tags.xml z pomocą modułu spark-xml do DataFrame'u
dfTags = spark.read.format('xml').option('rootTag', 'tags').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Tags.xml',schema=schemaTags)
dfTags.printSchema()
dfTags.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _Count: integer (nullable = true)
 |-- _ExcerptPostId: integer (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _TagName: string (nullable = true)
 |-- _WikiPostId: integer (nullable = true)

+------+--------------+---+--------------------+-----------+
|_Count|_ExcerptPostId|_Id|            _TagName|_WikiPostId|
+------+--------------+---+--------------------+-----------+
|    17|          5062|  1|                hops|       5061|
|    85|          7872|  2|             history|       7871|
|    69|          4880|  4|             brewing|       4879|
|    37|          5109|  5|             serving|       5108|
|    31|           304|  6|         temperature|        303|
|     4|          null|  7|            pilsener|       null|
|     4|          null|  9|         inebriation|       null|
|    15|          5122| 10|                 ale|       5121|
|    16|          5040| 11|               lager|       5039|
|    56|          7878| 12|           breweries|       7

In [77]:
#nadanie odpowiednich nazw kolumnom
dfTags1 = dfTags.withColumnRenamed("_Count","Count")\
                    .withColumnRenamed("_ExcerptPostId","PostId")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_TagName","TagName")\
                    .withColumnRenamed("_WikiPostId","WikiPostId")
dfTags1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Count: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- TagName: string (nullable = true)
 |-- WikiPostId: integer (nullable = true)

In [78]:
#zapisanie tabeli dfTags1 w formacie parquet
dfTags1.write.parquet('s3://stackexchangebeer1/parquet/tags.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK POSTHISTORY.XML__

In [11]:
#ustawienie odpowiednich typów danych dla pliku PostHistory.xml
from pyspark.sql.types import *

schemaPostHistory = StructType([
    StructField("_Comment", StringType(), True),
    StructField("_ContentLicense", StringType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_PostId", IntegerType(), True),
    StructField("_RevisionGUID", StringType(), True),
    StructField("_Text", StringType(), True),
    StructField("_UserDisplayName", StringType(), True),
    StructField("_UserId", IntegerType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#wczytanie pliku PostHistory.xml z pomocą modułu spark-xml do DataFrame'u
dfPostHistory = spark.read.format('xml').option('rootTag', 'posthistory').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/PostHistory.xml',schema=schemaPostHistory)
dfPostHistory.printSchema()
dfPostHistory.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _Comment: string (nullable = true)
 |-- _ContentLicense: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _PostId: integer (nullable = true)
 |-- _RevisionGUID: string (nullable = true)
 |-- _Text: string (nullable = true)
 |-- _UserDisplayName: string (nullable = true)
 |-- _UserId: integer (nullable = true)

+--------+---------------+--------------------+---+-------+--------------------+--------------------+----------------+-------+
|_Comment|_ContentLicense|       _CreationDate|_Id|_PostId|       _RevisionGUID|               _Text|_UserDisplayName|_UserId|
+--------+---------------+--------------------+---+-------+--------------------+--------------------+----------------+-------+
|    null|   CC BY-SA 3.0|2014-01-21 20:26:...|  1|      1|a17002a0-00b0-417...|I was offered a b...|            null|      7|
|    null|   CC BY-SA 3.0|2014-01-21 20:26:...|  2|      1|a17002a0-00b0-417...|What is a citra h...|    

In [13]:
#usunięcie kolumn RevisonGUID i ContentLicense
dfPostHistory1 = dfPostHistory.drop(dfPostHistory["_ContentLicense"]).drop(dfPostHistory["_RevisionGUID"])
dfPostHistory1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _Comment: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _PostId: integer (nullable = true)
 |-- _Text: string (nullable = true)
 |-- _UserDisplayName: string (nullable = true)
 |-- _UserId: integer (nullable = true)

In [18]:
#nadanie odpowiednich nazw kolumnom
dfPostHistory2 = dfPostHistory1.withColumnRenamed("_Comment","Comment")\
                    .withColumnRenamed("_CreationDate","Date")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_PostId","PostId")\
                    .withColumnRenamed("_Text","Text")\
                    .withColumnRenamed("_UserDisplayName","UserDisplayName")\
                    .withColumnRenamed("_UserId","UserId")
dfPostHistory2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Comment: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- UserId: integer (nullable = true)

In [23]:
from pyspark.sql.functions import split
#rozbicie kolumny CreationDate na 2 kolumny z datą oraz czasem utworzenia
dfPostHistory3 = dfPostHistory2.select("Comment", "Date", "Id", "PostId", "Text", "UserDisplayName", "UserId").withColumn('CreationDate',split(dfPostHistory2["Date"], ' ')[0])\
                                                                                                  .withColumn('Time',split(dfPostHistory2["Date"], ' ')[1])\
                                                                                                  .drop(dfPostHistory2["Date"])
#zmiana typu danych kolumny Date na DateType
dfPostHistory4= dfPostHistory3.withColumn("CreationDate", dfPostHistory3["CreationDate"].cast(DateType()))
dfPostHistory4.printSchema()
dfPostHistory4.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Comment: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Time: string (nullable = true)

+-------+---+------+--------------------+---------------+------+------------+------------+
|Comment| Id|PostId|                Text|UserDisplayName|UserId|CreationDate|        Time|
+-------+---+------+--------------------+---------------+------+------------+------------+
|   null|  1|     1|I was offered a b...|           null|     7|  2014-01-21|20:26:05.383|
|   null|  2|     1|What is a citra h...|           null|     7|  2014-01-21|20:26:05.383|
|   null|  3|     1|              <hops>|           null|     7|  2014-01-21|20:26:05.383|
|   null|  4|     2|As far as we know...|           null|     7|  2014-01-21|20:27:29.797|
|   null|  5|     2|When was the firs...|   

In [26]:
dfPostHistory4.select("Id","Text").show(20,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Text                                                                                                                                                                                                                                                                                                                                            |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
#oczyszczenie kolumny Text ze znaków \n\r
import pyspark.sql.functions as f
dfPostHistory5=dfPostHistory4.withColumn('Text',f.regexp_replace(f.regexp_replace('Text', '\r', ''),'\n', '')).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+------+--------------------+---------------+------+------------+------------+
|Comment| Id|PostId|                Text|UserDisplayName|UserId|CreationDate|        Time|
+-------+---+------+--------------------+---------------+------+------------+------------+
|   null|  1|     1|I was offered a b...|           null|     7|  2014-01-21|20:26:05.383|
|   null|  2|     1|What is a citra h...|           null|     7|  2014-01-21|20:26:05.383|
|   null|  3|     1|              <hops>|           null|     7|  2014-01-21|20:26:05.383|
|   null|  4|     2|As far as we know...|           null|     7|  2014-01-21|20:27:29.797|
|   null|  5|     2|When was the firs...|           null|     7|  2014-01-21|20:27:29.797|
|   null|  6|     2|           <history>|           null|     7|  2014-01-21|20:27:29.797|
|   null|  7|     3|How is low/no alc...|           null|     7|  2014-01-21|20:30:17.437|
|   null|  8|     3|How is reduced al...|           null|     7|  2014-01-21|20:30:17.437|

In [35]:
#zapisanie tabeli PostHistory w formacie parquet
dfPostHistory.write.parquet('s3://stackexchangebeer1/parquet/posthistory.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK COMMENTS.XML__

In [39]:
#ustawienie odpowiednich typów danych dla pliku Comments.xml
from pyspark.sql.types import *

schemaComments = StructType([
    StructField("_ContentLicense", StringType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_PostId", IntegerType(), True),
    StructField("_Score", IntegerType(), True),
    StructField("_Text", StringType(), True),
    StructField("_UserDisplayName", StringType(), True),
    StructField("_UserId", IntegerType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#wczytanie pliku Badges.xml z pomocą modułu spark-xml do DataFrame'u
dfComments = spark.read.format('xml').option('rootTag', 'comments').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Comments.xml',schema=schemaComments)
dfComments.printSchema()
dfComments.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _ContentLicense: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _PostId: integer (nullable = true)
 |-- _Score: integer (nullable = true)
 |-- _Text: string (nullable = true)
 |-- _UserDisplayName: string (nullable = true)
 |-- _UserId: integer (nullable = true)

+---------------+--------------------+---+-------+------+--------------------+----------------+-------+
|_ContentLicense|       _CreationDate|_Id|_PostId|_Score|               _Text|_UserDisplayName|_UserId|
+---------------+--------------------+---+-------+------+--------------------+----------------+-------+
|   CC BY-SA 3.0|2014-01-21 20:34:...|  1|      5|     0|This is a matter ...|            null|     10|
|   CC BY-SA 3.0|2014-01-21 20:52:...|  4|     16|     4|This question app...|            null|     39|
|   CC BY-SA 3.0|2014-01-21 20:55:...|  5|     16|     8|Obviously, you sh...|            null|     41|
|   CC BY-SA 3.0|2014-01-21 20:57:.

In [43]:
#usunięcie kolumny ContentLicense
dfComments1 = dfComments.drop('_ContentLicense')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#nadanie odpowiednich nazw kolumnom
dfComments2 = dfComments1.withColumnRenamed("_CreationDate","Date")\
                    .withColumnRenamed("_Id","Id")\
                    .withColumnRenamed("_PostId","PostId")\
                    .withColumnRenamed("_Score","Score")\
                    .withColumnRenamed("_Text","Text")\
                    .withColumnRenamed("_UserDisplayName","UserDisplayName")\
                    .withColumnRenamed("_UserId","UserId")
dfComments2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: timestamp (nullable = true)
 |-- Id: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- UserId: integer (nullable = true)

In [46]:
from pyspark.sql.functions import split
#rozbicie kolumny CreationDate na 2 kolumny z datą oraz czasem utworzenia
dfComments3 = dfComments2.select("Date", "Id", "PostId", "Score", "Text", "UserDisplayName", "UserId").withColumn('CreationDate',split(dfComments2["Date"], ' ')[0])\
                                                                                                  .withColumn('Time',split(dfComments2["Date"], ' ')[1])\
                                                                                                  .drop(dfComments2["Date"])
#zmiana typu danych kolumny Date na DateType
dfComments4= dfComments3.withColumn("CreationDate", dfComments3["CreationDate"].cast(DateType()))
dfComments4.printSchema()
dfComments4.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Time: string (nullable = true)

+---+------+-----+--------------------+---------------+------+------------+------------+
| Id|PostId|Score|                Text|UserDisplayName|UserId|CreationDate|        Time|
+---+------+-----+--------------------+---------------+------+------------+------------+
|  1|     5|    0|This is a matter ...|           null|    10|  2014-01-21| 20:34:28.74|
|  4|    16|    4|This question app...|           null|    39|  2014-01-21| 20:52:14.49|
|  5|    16|    8|Obviously, you sh...|           null|    41|  2014-01-21|20:55:44.723|
|  6|     5|    3|Doesn't this depe...|           null|    43|  2014-01-21|20:57:44.653|
|  7|    19|    4|I retracted my cl...|           null|    

In [47]:
dfComments4.select("Text").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
+---------------------------------------------------------------

In [51]:
#oczyszczenie kolumny Text ze znaków \n\r
dfComments5=dfComments4.withColumn('Text',f.regexp_replace(f.regexp_replace('Text', '\r', ''),'\n', ''))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
dfComments5.select("Text").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
+-------------------------------------------------------------------------------

In [53]:
#zapisanie tabeli Comments w formacie parquet
dfComments5.write.parquet('s3://stackexchangebeer1/parquet/comments.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__PLIK POSTS.XML__

In [62]:
#ustawienie odpowiednich typów danych dla pliku Posts.xml
from pyspark.sql.types import *

schemaPosts = StructType([
    StructField("_AcceptedAnswerId", IntegerType(), True),
    StructField("_AnswerCount", IntegerType(), True),
    StructField("_Body", StringType(), True),
    StructField("_ClosedDate", TimestampType(), True),
    StructField("_ComentCount", IntegerType(), True),
    StructField("_CommunityOwnedDate", TimestampType(), True),
    StructField("_ContentLicense", StringType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_FavoriteCount", IntegerType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_LastActivityDate", TimestampType(), True),
    StructField("_LastEditDate", TimestampType(), True),
    StructField("_LastEditorDisplayName", StringType(), True),
    StructField("_LastEditorUserId", IntegerType(), True),
    StructField("_OwnerDisplayName", StringType(), True),
    StructField("_OwnerUserId", IntegerType(), True),
    StructField("_ParentId", IntegerType(), True),
    StructField("_PostTypeId", IntegerType(), True),
    StructField("_ScoreId", IntegerType(), True),
    StructField("_Tags", StringType(), True),
    StructField("_Title", StringType(), True),
    StructField("_ViewCount", IntegerType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
#wczytanie pliku Posts.xml z pomocą modułu spark-xml do DataFrame'u
dfPosts = spark.read.format('xml').option('rootTag', 'posts').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Posts.xml',schema=schemaPosts)
dfPosts.printSchema()
dfPosts.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _AcceptedAnswerId: integer (nullable = true)
 |-- _AnswerCount: integer (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _ComentCount: integer (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _ContentLicense: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: integer (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: integer (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: integer (nullable = true)
 |-- _ParentId: integer (nullable = true)
 |-- _PostTypeId: integer (nullable = true)
 |-- _ScoreId: integer (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: integer (nullable = tru

In [95]:
#nadanie odpowednich nazw kolumn
dfPosts1 = dfPosts.withColumnRenamed("_AcceptedAnswerId", "AcceptedAnswerId")\
                    .withColumnRenamed("_AnswerCount", "AnswerCount")\
                    .withColumnRenamed("_Body", "Body")\
                    .withColumnRenamed("_ClosedDate", "CloseDate")\
                    .withColumnRenamed("_ComentCount", "ComentCount")\
                    .withColumnRenamed("_CommunityOwnedDate", "CommunityOwnedDate")\
                    .withColumnRenamed("_ContentLicense", "ContentLicense")\
                    .withColumnRenamed("_CreationDate", "Date")\
                    .withColumnRenamed("_FavoriteCount", "FavoriteCount")\
                    .withColumnRenamed("_Id", "Id")\
                    .withColumnRenamed("_LastActivityDate", "LastActivityDate")\
                    .withColumnRenamed("_LastEditDate", "LastEditDate")\
                    .withColumnRenamed("_LastEditorDisplayName", "LastEditorDisplayName")\
                    .withColumnRenamed("_LastEditorUserId", "LastEditorUserId")\
                    .withColumnRenamed("_OwnerDisplayName", "OwnerDisplayName")\
                    .withColumnRenamed("_OwnerUserId", "OwnerUserId")\
                    .withColumnRenamed("_ParentId", "ParentId")\
                    .withColumnRenamed("_PostTypeId", "PostTypeId")\
                    .withColumnRenamed("_ScoreId", "ScoreId")\
                    .withColumnRenamed("_AcceptedAnswerId", "AcceptedAnswerId")\
                    .withColumnRenamed("_Tags", "Tags")\
                    .withColumnRenamed("_Title", "Title")\
                    .withColumnRenamed("_ViewCount", "ViewCount")

dfPosts1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- CloseDate: timestamp (nullable = true)
 |-- ComentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- ScoreId: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)

In [96]:
#rozbicie kolumny CreationDate na 2 kolumny z datą oraz czasem utworzenia
dfPosts2 = dfPosts1.withColumn('CreationDate',split(dfPosts1["Date"], ' ')[0])\
                    .withColumn('Time',split(dfPosts1["Date"], ' ')[1])\
                    .drop(dfPosts1["Date"])

#zmiana typu danych kolumny Date na DateType
dfPosts3= dfPosts2.withColumn("CreationDate", dfPosts2["CreationDate"].cast(DateType()))

dfPosts3.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- CloseDate: timestamp (nullable = true)
 |-- ComentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- ScoreId: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Time: string (nullab

In [97]:
dfPosts3.select("Body").show(20,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [98]:
#oczyszczenie kolumny Body z niepotrzebnych znaków
dfPosts4=dfPosts3.withColumn('Body',f.regexp_replace(f.regexp_replace('Body', '\r', ''),'\n', ''))
dfPosts5=dfPosts4.withColumn('Body',f.regexp_replace(f.regexp_replace(f.regexp_replace(f.regexp_replace(f.regexp_replace(f.regexp_replace('Body', '<p>', ''),'</p>', ''),'<em>', ''),'</em>', ''),'<a>', ''),'</a>', ''))



dfPosts5.select("Body").show(10,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Body                                                                                                                                                                                                                                                                                                                                                                                                                            

In [107]:
#zapisanie tabeli Posts w formacie parquet
dfPosts5.write.parquet('s3://stackexchangebeer1/parquet/posts.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
path s3://stackexchangebeer1/parquet/posts.parquet already exists.
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1674289536756_0001/container_1674289536756_0001_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 885, in parquet
    self._jwrite.parquet(path)
  File "/mnt1/yarn/usercache/livy/appcache/application_1674289536756_0001/container_1674289536756_0001_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt1/yarn/usercache/livy/appcache/application_1674289536756_0001/container_1674289536756_0001_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: path s3://stackexchangebeer1/parquet/posts.parquet already exists.



__PLIK USERS.XML__

In [114]:
schemaUsers = StructType([
    StructField("_AboutMe", StringType(), True),
    StructField("_AccountId", IntegerType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_DisplayName", StringType(), True),
    StructField("_DownVotes", IntegerType(), True),
    StructField("_Id", IntegerType(), True),
    StructField("_LastAccesDate", TimestampType(), True),
    StructField("_Location", StringType(), True),
    StructField("_ProfileImageUrl", StringType(), True),
    StructField("_Reputation", IntegerType(), True),
    StructField("_UpVotes", IntegerType(), True),
    StructField("_Views", IntegerType(), True),
    StructField("_WebsiteUrl", StringType(), True),
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
#wczytanie pliku Users.xml z pomocą modułu spark-xml do DataFrame'u
dfUsers = spark.read.format('xml').option('rootTag', 'users').option('rowTag', 'row').load('s3://stackexchangebeer1/raw_data/Users.xml',schema=schemaUsers)
dfUsers.printSchema()
dfUsers.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _AboutMe: string (nullable = true)
 |-- _AccountId: integer (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _DisplayName: string (nullable = true)
 |-- _DownVotes: integer (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _LastAccesDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- _ProfileImageUrl: string (nullable = true)
 |-- _Reputation: integer (nullable = true)
 |-- _UpVotes: integer (nullable = true)
 |-- _Views: integer (nullable = true)
 |-- _WebsiteUrl: string (nullable = true)

+--------------------+----------+--------------------+--------------+----------+---+--------------+--------------------+--------------------+-----------+--------+------+--------------------+
|            _AboutMe|_AccountId|       _CreationDate|  _DisplayName|_DownVotes|_Id|_LastAccesDate|           _Location|    _ProfileImageUrl|_Reputation|_UpVotes|_Views|         _WebsiteUrl|
+--------------------+----------+--------------------

In [120]:
dfUsers1 = dfUsers.withColumnRenamed("_AboutMe", "AboutMe")\
                    .withColumnRenamed("_AccountId", "AccountId")\
                    .withColumnRenamed("_CreationDate", "Date")\
                    .withColumnRenamed("_DisplayName", "DisplayName")\
                    .withColumnRenamed("_DownVotes", "DownVotes")\
                    .withColumnRenamed("_Id", "Id")\
                    .withColumnRenamed("_LastAccesDate", "LastAccesDate")\
                    .withColumnRenamed("_Location", "Location")\
                    .withColumnRenamed("_ProfileImageUrl", "ProfileImageUrl")\
                    .withColumnRenamed("_Reputation", "Reputation")\
                    .withColumnRenamed("_UpVotes", "UpVotes")\
                    .withColumnRenamed("_Views", "Views")\
                    .withColumnRenamed("_WebsiteUrl", "WebsiteUrl")\

dfUsers1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- AboutMe: string (nullable = true)
 |-- AccountId: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- LastAccesDate: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- WebsiteUrl: string (nullable = true)

In [121]:
#rozbicie kolumny CreationDate na 2 kolumny z datą oraz czasem utworzenia
dfUsers2 = dfUsers1.withColumn('CreationDate',split(dfUsers1["Date"], ' ')[0])\
                    .withColumn('Time',split(dfUsers1["Date"], ' ')[1])\
                    .drop(dfUsers1["Date"])

#zmiana typu danych kolumny Date na DateType
dfUsers3= dfUsers2.withColumn("CreationDate", dfUsers2["CreationDate"].cast(DateType()))

dfUsers3.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- AboutMe: string (nullable = true)
 |-- AccountId: integer (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- LastAccesDate: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Time: string (nullable = true)

In [122]:
dfUsers3.select("CreationDate","Time").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+
|CreationDate|        Time|
+------------+------------+
|  2014-01-21|17:45:53.587|
|  2014-01-21|20:21:18.797|
|  2014-01-21| 20:22:19.07|
|  2014-01-21| 20:22:22.12|
|  2014-01-21|  20:22:24.3|
+------------+------------+
only showing top 5 rows

In [118]:
dfUsers3.select("AboutMe").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [136]:
#oczyszczenie kolumny tekstowej
dfUsers4=dfUsers3.withColumn('AboutMe',f.regexp_replace('AboutMe', '\\<p>|\\</p>|\\\n|\\\r|\\<n>|\\</n>|\\<a>|\\</a>|\\<li>|\\</li>|\\<strong>|\\</strong>|\\<code>|\\</code>', ''))
dfUsers4.select("AboutMe").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [137]:
#zapisanie tabeli Users w formacie parquet
dfUsers4.write.parquet('s3://stackexchangebeer1/parquet/users.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…